In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [12]:
## read pdfs from folder
loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [13]:
len(final_documents)

316

In [16]:
# Embedding  using Higgingface
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)


c:\Users\ps1477\Documents\projects\AI-study\RAG\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\ps1477\Documents\projects\AI-study\RAG\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ps1477\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develope

In [17]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568272e-02 -1.19099068e-02 -3.37892212e-02  2.94559114e-02
  5.19160256e-02  5.73840030e-02 -4.10018116e-02  2.74267606e-02
 -1.05128206e-01 -1.58055872e-02  7.94858783e-02  5.64318895e-02
 -1.31765716e-02 -3.41544449e-02  5.81600517e-03  4.72547635e-02
 -1.30746737e-02  3.12989787e-03 -3.44226249e-02  3.08406334e-02
 -4.09086272e-02  3.52737717e-02 -2.43761223e-02 -4.35830802e-02
  2.41503119e-02  1.31986486e-02 -4.84455097e-03  1.92347057e-02
 -5.43912649e-02 -1.42735094e-01  5.15529374e-03  2.93115713e-02
 -5.60810752e-02 -8.53529759e-03  3.14141288e-02  2.76736636e-02
 -2.06188671e-02  8.24231505e-02  4.15425412e-02  5.79654835e-02
 -3.71586867e-02  6.26162579e-03 -2.41390318e-02 -5.61797759e-03
 -2.51715723e-02  5.04976278e-03 -2.52801497e-02 -2.91943387e-03
 -8.24041571e-03 -5.69603927e-02  2.30822600e-02 -5.54215023e-03
  5.11555411e-02  6.09937273e-02  6.49766028e-02 -5.38513996e-02
  2.19109710e-02 -2.54194979e-02 -4.49223891e-02  4.22458909e-02
  4.75252643e-02  7.23222

In [18]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [19]:
#query using similarity search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_documents=vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [21]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020F65533290> search_kwargs={'k': 3}


In [ ]:
# import os
# huggingface_api_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")

import os
os.environ['HUGGINGFACEHUB_API_TOKEN']="token"

In [51]:
# print(huggingface_api_key)


The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [52]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="google/gemma-2-27b",
    model_kwargs={"temperature":0.1,"max_length":500}
)

GatedRepoError: 403 Client Error. (Request ID: Root=1-6683edd4-7544dc1028518206415697e6;f3d18453-320a-4b2d-a2c0-d10c620b4ac4)

Cannot access gated repo for url https://huggingface.co/api/models/google/gemma-2-27b.
Access to model google/gemma-2-27b is restricted and you are not in the authorized list. Visit https://huggingface.co/google/gemma-2-27b to ask for access.